In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# Read your CSV
df = pd.read_csv('Udemy Courses.csv')

# Connect to Supabase
conn = psycopg2.connect(
    host="db.imarcyzsoqegmsthvika.supabase.co",
    database="postgres",
    user="postgres",
    password="Aman2004",
    port="5432"
)

cursor = conn.cursor()

# Insert data
columns = df.columns.tolist()
values = [tuple(x) for x in df.to_numpy()]

insert_query = f"INSERT INTO courses ({','.join(columns)}) VALUES %s"
execute_values(cursor, insert_query, values)

conn.commit()
cursor.close()
conn.close()

print("Data imported successfully!")

print("DATABASE VERIFICATION")

# Test queries
test_queries = [
    ("Total Courses", "SELECT COUNT(*) FROM udemy_courses"),
    ("Total Subjects", "SELECT COUNT(DISTINCT subject) FROM udemy_courses"),
    ("Paid Courses", "SELECT COUNT(*) FROM udemy_courses WHERE is_paid = 1"),
    ("Free Courses", "SELECT COUNT(*) FROM udemy_courses WHERE is_paid = 0"),
]

for name, query in test_queries:
    result = pd.read_sql_query(query, conn)
    print(f"{name}: {result.iloc[0, 0]}")

print("KEY INSIGHTS FROM SQL")

# 1. Top subjects
print("\n1. TOP 4 SUBJECTS BY SUBSCRIBERS:")
query = """
SELECT subject, SUM(num_subscribers) as total_subscribers
FROM udemy_courses
GROUP BY subject
ORDER BY total_subscribers DESC
"""
result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

# 2. Pricing analysis
print("\n2. AVERAGE PRICE BY SUBJECT:")
query = """
DECLARE @avg_subscribers DECIMAL(18,2);
SELECT @avg_subscribers = AVG(CAST(num_subscribers AS DECIMAL))
FROM udemy_courses;

SELECT 
    subject,
    COUNT(*) as total_courses,
    ROUND(AVG(CASE 
        WHEN num_subscribers > @avg_subscribers 
        THEN price 
        ELSE NULL 
    END), 2) as optimal_price,
    ROUND(AVG(price), 2) as overall_avg_price
FROM udemy_courses
WHERE is_paid = 1
GROUP BY subject
HAVING COUNT(*) >= 10
ORDER BY optimal_price DESC;
"""
result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

# 3. Engagement leaders
print("\n3. TOP 20 MOST ENGAGING COURSES:")
query = """
SELECT TOP 20
    course_title,
    subject,
    num_subscribers,
    num_reviews,
    ROUND(review_rate, 4) as engagement_rate,
    price,
    quality_score
FROM udemy_courses
WHERE num_subscribers > 500
ORDER BY review_rate DESC
"""
result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

# 4. Course length analysis
print("\n4. COURSE LENGTH VS POPULARITY:")
query = """
SELECT 
    length_category,
    COUNT(*) as courses,
    ROUND(AVG(CAST(num_subscribers AS DECIMAL)), 0) as avg_subscribers,
    ROUND(AVG(review_rate), 4) as avg_engagement,
    ROUND(AVG(price), 2) as avg_price,
    ROUND(AVG(popularity_score), 4) as avg_popularity
FROM udemy_courses
GROUP BY length_category
ORDER BY avg_subscribers DESC
"""
result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

# 5. Yearly trends
print("\n5. YEARLY TRENDS:")
query = """
SELECT 
    CASE 
        WHEN published_year >= 2016 THEN '2016-2017 (Recent)'
        WHEN published_year >= 2014 THEN '2014-2015'
        WHEN published_year >= 2012 THEN '2012-2013'
        WHEN published_year >= 2010 THEN '2010-2011'
        ELSE '2009 or Earlier'
    END as course_age_group,
    COUNT(*) as courses,
    ROUND(AVG(CAST(num_subscribers AS DECIMAL)), 0) as avg_subscribers,
    ROUND(AVG(review_rate), 4) as avg_engagement,
    ROUND(AVG(popularity_score), 4) as avg_popularity
FROM udemy_courses
WHERE published_year IS NOT NULL
GROUP BY 
    CASE 
        WHEN published_year >= 2016 THEN '2016-2017 (Recent)'
        WHEN published_year >= 2014 THEN '2014-2015'
        WHEN published_year >= 2012 THEN '2012-2013'
        WHEN published_year >= 2010 THEN '2010-2011'
        ELSE '2009 or Earlier'
    END
ORDER BY courses DESC
"""
result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

print("EXPORTING DATA FOR VISUALIZATION")

# Export views
views_to_export = [
    'vw_subject_performance',
    'vw_top_courses',
    'vw_pricing_analysis',
    'vw_length_impact',
    'vw_paid_vs_free',
    'vw_yearly_trends',
    'vw_best_value_courses'
]

for view_name in views_to_export:
    try:
        query = f"SELECT * FROM {view_name}"
        df_export = pd.read_sql_query(query, conn)
        csv_name = view_name.replace('vw_', '') + '.csv'
        df_export.to_csv(csv_name, index=False)
        print(f"Exported {csv_name} ({len(df_export)} rows)")
    except Exception as e:
        print(f"Could not export {view_name}: {e}")

# Export main table
query = "SELECT * FROM udemy_courses"
df_main = pd.read_sql_query(query, conn)
df_main.to_csv('courses_for_tableau.csv', index=False)
print(f"Exported courses_for_tableau.csv ({len(df_main)} rows)")

conn.close()


Setting up SQL Server database connection...
Connected to SQL Server successfully!
DATABASE VERIFICATION
Total Courses: 3678
Total Subjects: 4
Paid Courses: 3368
Free Courses: 310
KEY INSIGHTS FROM SQL

1. TOP 4 SUBJECTS BY SUBSCRIBERS:
            subject  total_subscribers
    Web Development          7980572.0
   Business Finance          1868711.0
     Graphic Design          1063148.0
Musical Instruments           846689.0

2. AVERAGE PRICE BY SUBJECT:
            subject  total_courses  optimal_price  overall_avg_price
Musical Instruments            634         109.62              53.15
     Graphic Design            568         103.98              61.39
    Web Development           1067         102.17              86.64
   Business Finance           1099          96.22              74.54

3. TOP 20 MOST ENGAGING COURSES:
                                                course_title          subject  num_subscribers  num_reviews  engagement_rate  price  quality_score
 Complete GS

C:\Users\amanr\AppData\Local\Temp\ipykernel_10568\1137951483.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, conn)
C:\Users\amanr\AppData\Local\Temp\ipykernel_10568\1137951483.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, conn)
C:\Users\amanr\AppData\Local\Temp\ipykernel_10568\1137951483.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, conn)
C:\Users\amanr\AppData\Local\Temp\ipykernel_10568\1137951483.py:86:

Exported courses_for_tableau.csv (3678 rows)
